In [ ]:
import os
import pandas as pd

year = input("Enter year (e.g. 2025): ").strip()
quarter = input("Enter quarter (Q1, Q2, Q3, Q4): ").strip().upper()

BASE_DIR = os.path.join(
    "C:\\Users",
    os.environ.get("USERNAME"),
    "Documents",
    "Equity"
)

HOME = os.path.join(BASE_DIR, f"{year}_{quarter}")
DATA_DIR = os.path.join(HOME, "data")

print("Reading data from:", DATA_DIR)


In [ ]:
cashflows = pd.read_parquet(os.path.join(DATA_DIR, "cashflows.parquet"))
kmp = pd.read_parquet(os.path.join(DATA_DIR, "kmp.parquet"))
grades = pd.read_parquet(os.path.join(DATA_DIR, "grades.parquet"))

print("Loaded:")
print(f"cashflows: {cashflows.shape}")
print(f"kmp:        {kmp.shape}")
print(f"grades:     {grades.shape}")


In [ ]:
# Ensure quarter is numeric 1-4 (adjust if yours is 'Q1' style)
# cashflows['Quarter of Transaction Date'] should already be 1..4 based on your SQL

cashflows["q_idx"] = cashflows["Year of Transaction Date"] * 4 + cashflows["Quarter of Transaction Date"]

# If grades uses different column names, align them (edit as needed)
# grades = grades.rename(columns={"Year": "Year of Transaction Date", "Quarter": "Quarter of Transaction Date"})
grades["q_idx"] = grades["Year of Transaction Date"] * 4 + grades["Quarter of Transaction Date"]


In [ ]:
first_pgrade_by_fund = (
    cashflows
    .sort_values(["FundID", "q_idx"])
    .groupby("FundID")["First Grading-P"]
    .apply(lambda s: s.dropna().iloc[0] if s.notna().any() else pd.NA)
)

In [ ]:
df = cashflows.merge(
    grades[["FundID", "Year of Transaction Date", "Quarter of Transaction Date", "Grade"]],
    on=["FundID", "Year of Transaction Date", "Quarter of Transaction Date"],
    how="left"
)


In [ ]:
# Merge grades onto cashflows timeline
df = cashflows.merge(
    grades[["FundID", "Year of Transaction Date", "Quarter of Transaction Date", "Grade"]],
    on=["FundID", "Year of Transaction Date", "Quarter of Transaction Date"],
    how="left"
)

# Build quarter index and sort
df["q_idx"] = df["Year of Transaction Date"] * 4 + df["Quarter of Transaction Date"]
df = df.sort_values(["FundID", "q_idx"]).copy()

# Fund-level seed: first available P-grade from cashflows
first_pgrade_by_fund = (
    cashflows
    .sort_values(["FundID", "q_idx"])
    .groupby("FundID")["First Grading-P"]
    .apply(lambda s: s.dropna().iloc[0] if s.notna().any() else pd.NA)
)

# 1) Forward-fill grades within each fund (fills gaps in-between + trailing)
df["Grade"] = df.groupby("FundID")["Grade"].ffill()

# 2) Fill ONLY the beginning (leading NaNs) using first P-grade
# (after ffill, the only NaNs left are at the start of each fund, or funds with no grades at all)
df["Grade"] = df["Grade"].fillna(df["FundID"].map(first_pgrade_by_fund))


In [ ]:
df = df.merge(
    kmp,
    on="FundID",
    how="left",
    suffixes=("", "_kmp")
)

In [ ]:
cols_to_keep = [
    'VC Fund Name', 'VC Fund Status',
    'Vintage Year',
    'Year of Transaction Date', 'Quarter of Transaction Date',
    'FundID',
    'Fund Workflow Stage', 'Investment Strategy Set Up',
    'Adj Strategy', 'Main Sector', 'Stage Focus',
    'Target Fund Size', 'Team Location', 'New Team',
    'First Closing Date', 'Final Closing Date',
    'Planned end date with add. years as per legal doc',
    'Fund Currency',
    'Transaction Quarter', 'Commitment EUR',
    'Signed Amount EUR',
    'Adj Drawdown EUR', 'Adj Repayment EUR', 'Recallable',
    'NAV Adjusted EUR', 'Grade',
    'Recallable_Percentage_Decimal', 'Expiration_Quarters'
]


In [ ]:
# Keep only columns that actually exist
final_cols = [c for c in cols_to_keep if c in df.columns]

# Optional: warn if something is missing
missing_cols = set(cols_to_keep) - set(final_cols)
if missing_cols:
    print("Warning: missing columns:", missing_cols)

data = df[final_cols]


In [ ]:
# Parse Vintage Year as datetime
data["Vintage Year"] = pd.to_datetime(data["Vintage Year"], errors="coerce")

# Keep only vintage year after 2004
data = data[data["Vintage Year"].dt.year > 2004].copy()
data["Vintage Year"].dt.year.min(), data["Vintage Year"].dt.year.max()

In [ ]:
#Remove Funds with no grade
data = data[data["Grade"].notna()].copy()

In [ ]:
#set Nav values constraint at 0

data.loc[data["NAV Adjusted EUR"] < 0, "NAV Adjusted EUR"] = 0

In [ ]:
#Create Age in columns
# Ensure datetime type
data["First Closing Date"] = pd.to_datetime(data["First Closing Date"], errors="coerce")

# Build quarter index for transaction date
data["tx_q_idx"] = data["Year of Transaction Date"] * 4 + data["Quarter of Transaction Date"]

# Build quarter index for first closing date
data["fc_q_idx"] = data["First Closing Date"].dt.year * 4 + data["First Closing Date"].dt.quarter

# Fund age in quarters
data["Fund_Age_Quarters"] = data["tx_q_idx"] - data["fc_q_idx"]


In [ ]:
# Take care of negative Drawdons

draw_col = "Adj Drawdown EUR"
lookback_quarters = 20  # 5 years

# Ensure numeric
data[draw_col] = pd.to_numeric(data[draw_col], errors="coerce").fillna(0)

# Sort by fund and time
data = data.sort_values(["FundID", "tx_q_idx"]).copy()

for fund_id, grp in data.groupby("FundID"):
    idx = grp.index.tolist()

    for pos, i in enumerate(idx):
        val = data.at[i, draw_col]

        if val < 0:
            refund = -val
            data.at[i, draw_col] = 0

            # walk backwards up to 20 quarters
            start = max(0, pos - lookback_quarters)
            for j in range(pos - 1, start - 1, -1):
                prev_idx = idx[j]
                prev_val = data.at[prev_idx, draw_col]

                if refund <= 0:
                    break

                if prev_val > 0:
                    take = min(prev_val, refund)
                    data.at[prev_idx, draw_col] = prev_val - take
                    refund -= take


In [ ]:
#Capacity and Drawdown ratio

fund_col = "FundID"
time_col = "tx_q_idx"

draw_col = "Adj Drawdown EUR"   # draw_i,t
commit_flow_col = "Commitment EUR"
rc_flow_col = "Recallable"

# Ensure numeric
for c in [draw_col, commit_flow_col, rc_flow_col]:
    data[c] = pd.to_numeric(data[c], errors="coerce").fillna(0)

# Sort for correct cumulative logic
data = data.sort_values([fund_col, time_col]).copy()

# Commitment level at t (handles commitments at different times)
data["commit_cum_t"] = data.groupby(fund_col)[commit_flow_col].cumsum()

# Recallable capacity level at t (use cumsum if Recallable is a flow per quarter)
data["rc_cum_t"] = data.groupby(fund_col)[rc_flow_col].cumsum()

# Cumulative drawdown up to previous quarter (t-1)
data["draw_cum_prev"] = (
    data.groupby(fund_col)[draw_col].cumsum().shift(1).fillna(0)
)

# Capacity before draw: commitment_i,t + RC_i,t - draw_cum_{t-1}, floored at 0
data["Capacity"] = (data["commit_cum_t"] + data["rc_cum_t"] - data["draw_cum_prev"]).clip(lower=0)

# Drawdown ratio rD = draw / capacity (only when capacity > 0)
data["Drawdown_Ratio"] = 0.0
m = data["Capacity"] > 0
data.loc[m, "Drawdown_Ratio"] = data.loc[m, draw_col] / data.loc[m, "Capacity"]

# Optional: keep within [0,1]
data["Drawdown_Ratio"] = data["Drawdown_Ratio"].clip(lower=0, upper=1)

helper_cols = [
    "commit_cum_t",
    "rc_cum_t",
]

data = data.drop(columns=[c for c in helper_cols if c in data.columns])



In [ ]:
# Repayment Ratio

repay_col = "Adj Repayment EUR"
nav_col = "NAV Adjusted EUR"

# Ensure numeric
data[repay_col] = pd.to_numeric(data[repay_col], errors="coerce").fillna(0)
data[nav_col] = pd.to_numeric(data[nav_col], errors="coerce").fillna(0)

# Floor negative repayments to 0 (if that’s your rule)
data.loc[data[repay_col] < 0, repay_col] = 0

# Make sure we're sorted by fund-quarter
data = data.sort_values(["FundID", "tx_q_idx"]).copy()

# Previous quarter NAV (within fund)
data["NAV_Adj_EUR_prev_q"] = data.groupby("FundID")[nav_col].shift(1)

# Treat "very close to 0" NAV as 0
NAV_EPS = 100  # adjust if you want (e.g., 1, 10, 100 EUR depending on your scale)

data["Repayment_Ratio"] = 0.0
m = data["NAV_Adj_EUR_prev_q"].fillna(0).abs() > NAV_EPS
data.loc[m, "Repayment_Ratio"] = data.loc[m, repay_col] / data.loc[m, "NAV_Adj_EUR_prev_q"].abs()

# Floor at 0 (should already be non-negative)
data["Repayment_Ratio"] = data["Repayment_Ratio"].clip(lower=0)

helper_cols = ["NAV_Adj_EUR_prev_q", "tx_q_idx", "fc_q_idx"]

data = data.drop(columns=[c for c in helper_cols if c in data.columns])

# Identify FundIDs to remove (any quarter with Repayment_Ratio > 1)
funds_to_remove = (
    data.loc[data["Repayment_Ratio"] > 1, "FundID"]
    .dropna()
    .unique()
)

# Count how many funds will be removed
n_funds_removed = len(funds_to_remove)

# Remove those FundIDs completely
data = data[~data["FundID"].isin(funds_to_remove)].copy()

print(f"Removed {n_funds_removed} FundID(s) with Repayment_Ratio > 1.")


In [ ]:
merged_csv = os.path.join(DATA_DIR, "data.csv")
merged_parquet = os.path.join(DATA_DIR, "data.parquet")

data.to_csv(
    merged_csv,
    index=False,
    sep=";",              # Excel-friendly
    encoding="utf-8-sig"
)

data.to_parquet(merged_parquet, index=False)

print("Filtered merged dataset saved:")
print(merged_csv)
print(merged_parquet)


In [ ]:
strategy_col = "Adj Strategy"
grade_col = "Grade"

# Convert quarters → years
data["Fund_Age_Years"] = pd.to_numeric(
    data["Fund_Age_Quarters"], errors="coerce"
) / 4

# Define buckets to match downstream script EXACTLY
bins = [0, 2, 4, 6, 8, 10, 15, 20, float("inf")]
labels = ["0-2y", "2-4y", "4-6y", "6-8y", "8-10y", "10-15y", "15-20y", "20y+"]

data["Age_Bucket"] = pd.cut(
    data["Fund_Age_Years"],
    bins=bins,
    labels=labels,
    right=False   # [0,2), [2,4), ...
)


In [ ]:
counts_s_g_age = (
    data
    .dropna(subset=[strategy_col, grade_col, "Age_Bucket"])
    .groupby([strategy_col, grade_col, "Age_Bucket"])
    .size()
    .reset_index(name="Datapoints")
    .sort_values([strategy_col, grade_col, "Age_Bucket"])
)

print(counts_s_g_age)


In [ ]:
pivot_s_g_age = (
    data
    .dropna(subset=[strategy_col, grade_col, "Age_Bucket"])
    .pivot_table(
        index=[strategy_col, grade_col],
        columns="Age_Bucket",
        values="FundID",
        aggfunc="count",
        fill_value=0
    )
)

print(pivot_s_g_age)


In [ ]:
# ============================================================
# 10-year (40-quarter) Migration Matrices for VC and PE
# ============================================================

import numpy as np
import pandas as pd
import os

H_YEARS = 10
H_Q = H_YEARS * 4  # 40 quarters

strategy_filter = ["Venture Capital", "Private Equity"]
strategy_col = "Adj Strategy"
fund_col = "FundID"
time_col = "tx_q_idx"          # quarter index already in your data earlier
grade_col = "Grade"

# Ensure we still have tx_q_idx (your script drops it later)
# If tx_q_idx was dropped, reconstruct it from year/quarter columns:
if time_col not in data.columns:
    data[time_col] = data["Year of Transaction Date"] * 4 + data["Quarter of Transaction Date"]

# Keep only relevant rows
mig = data.loc[data[strategy_col].isin(strategy_filter), [fund_col, time_col, grade_col, strategy_col]].copy()

# Clean grades (optional: keep only A/B/C/D if you want)
# If grades are already A/B/C/D, this is fine.
mig[grade_col] = mig[grade_col].astype(str).str.strip()

# Sort within fund time
mig = mig.sort_values([fund_col, time_col])

# Map (FundID, tx_q_idx) -> Grade, then join future grade at +40 quarters
mig["future_q_idx"] = mig[time_col] + H_Q

future = mig[[fund_col, time_col, grade_col]].copy()
future = future.rename(columns={time_col: "future_q_idx", grade_col: "Grade_tplus10y"})

mig = mig.merge(future, on=[fund_col, "future_q_idx"], how="left")

# Keep only rows where we have a 10y-ahead grade
mig = mig.dropna(subset=["Grade_tplus10y"]).copy()

# Function to build migration matrices
def migration_matrix(df_sub: pd.DataFrame, from_col="Grade", to_col="Grade_tplus10y"):
    # counts
    counts = pd.crosstab(df_sub[from_col], df_sub[to_col])
    # row-normalized probabilities
    probs = counts.div(counts.sum(axis=1), axis=0)
    return counts, probs

results = {}

for strat in strategy_filter:
    sub = mig[mig[strategy_col] == strat].copy()
    if sub.empty:
        print(f"\nNo 10-year transitions available for: {strat}")
        continue

    counts, probs = migration_matrix(sub, from_col=grade_col, to_col="Grade_tplus10y")
    results[strat] = (counts, probs)

    print(f"\n==============================")
    print(f"10Y Migration Matrix — {strat}")
    print(f"Transitions counted: {len(sub)}")
    print(f"From (rows) -> To (cols) over {H_YEARS} years")
    print(f"==============================\n")

    print("Counts:")
    print(counts)
    print("\nProbabilities (row-normalized):")
    print(probs.round(4))

# Optional: save to Excel-friendly CSVs
out_dir = DATA_DIR
for strat, (counts, probs) in results.items():
    safe = strat.replace(" ", "_").replace("/", "_")
    counts_path = os.path.join(out_dir, f"migration_{safe}_{H_YEARS}y_counts.csv")
    probs_path  = os.path.join(out_dir, f"migration_{safe}_{H_YEARS}y_probs.csv")

    counts.to_csv(counts_path, sep=";", encoding="utf-8-sig")
    probs.to_csv(probs_path, sep=";", encoding="utf-8-sig")

    print(f"\nSaved {strat} migration outputs:")
    print(counts_path)
    print(probs_path)
